In [ ]:
#Spark check
import findspark
findspark.init()
import time
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col
from pyspark import SparkConf

import pyspark
from pyspark.sql import SQLContext, HiveContext

conf = SparkConf().setAppName("my_app")
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)


In [ ]:
#Checking timestamp comparison for spark dataframe

prev_row={}
pixel={}
cur_row={}
prev_row['performance_timeobj'] = str(datetime.strptime('2017-02-01 23:40:00.0', "%Y-%m-%d %H:%M:%S.0"))
pixel['order_timestamp'] = datetime.strptime('2017-02-02 13:37:00', "%Y-%m-%d %H:%M:%S")
cur_row['performance_timeobj'] = datetime.strptime('2017-02-02 00:36:34.0', "%Y-%m-%d %H:%M:%S.0")
delta = -timedelta(hours=16)
print prev_row['performance_timeobj'], cur_row['performance_timeobj']
print type(prev_row['performance_timeobj']), type(cur_row['performance_timeobj'])
# if (prev_row['performance_timeobj'] + delta)  < pixel['order_timestamp'] <= (cur_row['performance_timeobj'] + delta):
#     print 'yipee'
# else:
#     print 'noo'

In [ ]:
#Spark datetime data types

a = '2017-02-01 23:40:00'
da = datetime.strptime('2017-02-01 23:40:00', "%Y-%m-%d %H:%M:%S")
df = sqlContext.createDataFrame([(da,da),(da,da)],['dt1','dt2'])
df.show()
print df.take(1)[0], type(df.take(1)[0]['dt1']) 
print df.printSchema()

#udf_timestamp = udf(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S"), TimestampType())
udf_timestamp = udf(lambda x : x, TimestampType())
df = df.withColumn("dt1", udf_timestamp("dt1"))
print df.printSchema()
print df.show()

In [ ]:
a = '2017-02-01 23:40:00'
#print TimestampType

from pyspark.sql import Row
zz = Row(b=1) + Row(a=2)
print type(zz)
print sc.parallelize([zz]).toDF(['b','a']).show()
#zz2 = Row(b=1)
#print zz2.toDF(['b'])
z = Row(b=2)
print z['b']

zz = Row(b=1)
print zz, type(zz)
print zz[0]

In [ ]:
import pandas as pd
a = '2017-02-01 23:40:00'
b = '2017-02-02 23:40:00'
at = pd.to_datetime(a)
bt = pd.to_datetime(b)
print pd.to_datetime(a), type(pd.to_datetime(a))
print at > bt
print a, at
print dir(at)

In [ ]:
#Pyspark logging
# print sc.applicationId() #sample tracking url : http://yn0004.dev.abc:8088/proxy/application_1483331339927_2935/
# print sc._jsc.sc().uiWebUrl().get()
# log4jLogger = sc._jvm.org.apache.log4j
# LOGGER = log4jLogger.LogManager.getLogger(__name__)
# LOGGER.info("pyspark script logger initialized")

#Pyspark logging
# print sc._jsc.sc().uiWebUrl().get()
# log4jLogger = sc._jvm.org.apache.log4j
# LOGGER = log4jLogger.LogManager.getLogger(__name__)
# LOGGER.info("pyspark script logger initialized")

log4j = sc._jvm.org.apache.log4j
log4j.LogManager.getRootLogger().setLevel(log4j.Level.ERROR)

In [ ]:
#Experimenting with dataframe partitioning
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
b=sqlContext.createDataFrame(sc.parallelize([(1,1,1,10), (1,1,1,2), (1,1,2,2), (2,1,2,2), (2,1,2,1), (1,2,1,1), (1,2,1,2), (1,1,2,1) ]),['a','b','c','d'])
#b.repartition('a','b','c').orderBy(['a','b','c']).show()
#b.repartition('c').show()
# print b.show()
# print b.repartition('a').show()
# print b.repartition('a').rdd.getNumPartitions()
# print b.repartition('a').sortWithinPartitions('d').show()
# print b.repartition('a','b','c').show()
# print b.repartition('a','b','c').rdd.getNumPartitions()
# print b.repartition('a','b','c').rdd.glom().collect()
#print b.orderBy(['c','b','a']).show()
#print b.orderBy(['c','b','a']).rdd.getNumPartitions()
#repartition and orderby are not required at same time as order is same as sort and will reshuffle
#print b.repartition('a','b','c').orderBy(['c','b','a']).show()
#print b.repartition('a','b','c').orderBy(['c','b','a']).rdd.getNumPartitions()
# tmp = b.orderBy(['c','b','a']).sortWithinPartitions('d')
# print b.orderBy(['c','b','a']).rdd.getNumPartitions()
# print b.orderBy(['c','b','a']).rdd.glom().collect()
# print b.orderBy(['c','b','a']).sortWithinPartitions('d').show()
print 'check:'
#tmp = b.orderBy(['c','b']).sortWithinPartitions(['a','d'])
#tmp = b.orderBy(['a']).sortWithinPartitions(['a','b','c','d'])
tmp = b.orderBy(['c']).sortWithinPartitions(['c','b','a','d'],ascending=[True,True,True,False])
print tmp.show()
print tmp.rdd.getNumPartitions()
print tmp.rdd.glom().collect()
#print b.orderBy(['c','b','a']).sortWithinPartitions('d').rdd.getNumPartitions()
#print b.orderBy(['c','b','a']).sort('d').show()
#print b.orderBy(['c','b','a']).sort('d').rdd.getNumPartitions()

#http://stackoverflow.com/questions/33831561/pyspark-repartition-vs-partitionby

In [ ]:
#Experimenting with rdd partitioning

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
b=sqlContext.createDataFrame(sc.parallelize([(1,1,1,1), (1,1,1,2), (1,1,2,2), (1,1,2,1), (2,1,2,2), (2,1,2,1), (1,2,1,1), (1,2,1,2) ]),['a','b','c','d'])

print ord_test_df.rdd.map(lambda x : (x['p_performance_campaignId'], x)).partitionBy(840).distinct().map(lambda x : x[0]).glom().collect()

In [ ]:
#Dataframe creation

sqlContext = SQLContext(sc)
a = {'q':1,'b':2,'d':12}
b = {'qq':1,'b':2,'c':3}
b=sqlContext.createDataFrame(sc.parallelize([a,b]))
c = b.withColumn('ab',b['b']+2)
d = c.withColumn('ab',c['ab']+1)
print d.show()
# c = sc.parallelize([a,b])
# print type(c), dir(c)

In [ ]:
#Time zone
#http://stackoverflow.com/questions/18812638/get-timezone-used-by-datetime-datetime-fromtimestamp
#http://stackoverflow.com/questions/3168096/getting-computers-utc-offset-in-python
print dir(datetime)
print datetime.fromtimestamp(float(1483540473910)/1000) #Gives diff result on diff machines
print datetime.utcfromtimestamp(float(1483540473910)/1000), datetime.utcfromtimestamp(float(1483540473910)/1000) - timedelta(hours=8) 
#do = datetime.fromtimestamp(float(1483540473910)/1000)
#print datetime.utcoffset(do)
a, b = 1.481702427E9, 1.481673627E9
print datetime.fromtimestamp(a), datetime.fromtimestamp(b) 
print datetime.utcfromtimestamp(a), datetime.utcfromtimestamp(b)
##Result on local laptop
# 2016-12-14 00:00:27 2016-12-13 16:00:27
# 2016-12-14 08:00:27 2016-12-14 00:00:27
##Below is result on some server
# >>> a, b = 1.481702427E9, 1.481673627E9
# >>> print datetime.fromtimestamp(a), datetime.fromtimestamp(b) 
# 2016-12-14 08:00:27 2016-12-14 00:00:27
# >>> print datetime.utcfromtimestamp(a), datetime.utcfromtimestamp(b)
# 2016-12-14 08:00:27 2016-12-14 00:00:27

In [ ]:
#Access hdfs avro data
# run this when in JupyterHub
import os
import findspark
findspark.init('/usr/lib/spark/')

from pyspark import SparkContext 
from pyspark.sql import SQLContext 
from pyspark.sql import HiveContext
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)

df = sqlContext.sql("CREATE TEMPORARY TABLE table_name USING com.databricks.spark.avro OPTIONS (path 'hdfs:///user/hadoop/a/b.avro')")

df = sqlContext.sql("SELECT a,b,c FROM table_name")

#Window function links
https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html
http://stackoverflow.com/questions/38414768/pyspark-data-frame-api-filter-rows-where-the-lead-lag-are-specific-values-wind
http://stackoverflow.com/questions/33207164/spark-window-functions-rangebetween-dates
http://stackoverflow.com/questions/36725353/applying-a-window-function-to-calculate-differences-in-pyspark

In [ ]:
#Window functions
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, sum, col
from pyspark.sql import functions as F
from pyspark.sql import HiveContext
from datetime import datetime
import sys

hiveContext = HiveContext(sc)

customers = sc.parallelize([(1234, 1234, datetime.strptime('2016-02-02 00:01:12', "%Y-%m-%d %H:%M:%S"), 1, 110, 1),
                            (1234, 1234, datetime.strptime('2016-02-02 02:02:12', "%Y-%m-%d %H:%M:%S"), 1, 111, 2),
                            (1234, 1234, datetime.strptime('2016-02-02 03:02:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 3),
                            (1235, 1235, datetime.strptime('2016-02-02 01:02:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 5),
                            (1235, 1235, datetime.strptime('2016-02-02 02:02:12', "%Y-%m-%d %H:%M:%S"), 1, 113, 10),
                            (1236, 1236, datetime.strptime('2016-02-02 01:02:12', "%Y-%m-%d %H:%M:%S"), 1, 114, 14),
                            (1236, 1236, datetime.strptime('2016-02-02 03:02:12', "%Y-%m-%d %H:%M:%S"), 1, 115, 22)])
df = hiveContext.createDataFrame(customers, ["performance_campaignId", "performance_keywordId", "performance_timeobj", "bq_resale_orders", "bq_resale_gtv", "performance_cost"])
print df.printSchema()

partitioning_list = ["performance_campaignId","performance_keywordId"]

#Moving average
wSpec1 = Window.partitionBy(partitioning_list).orderBy("performance_timeobj").rowsBetween(-1, 0)
print df.withColumn("movingAvg", avg(df["bq_resale_gtv"]).over(wSpec1)).show()

#cumulative sum
wSpec2 = Window.partitionBy(partitioning_list).orderBy("performance_timeobj").rowsBetween(-sys.maxint, 0) #Figure out Long.MinValue
print df.withColumn("cumulativeSum", sum(df["bq_resale_orders"]).over(wSpec2)).show()

#Filtered date windowed query
wSpec3 = Window.partitionBy(partitioning_list).orderBy(col("performance_timeobj").cast("timestamp").cast("long")).rangeBetween(-1 * 3600, 0)
print df.withColumn("cumulativeDateFilteredSum", sum(df["bq_resale_orders"]).over(wSpec3)).show()

#Testing lag
wSpec4 = Window.partitionBy(partitioning_list).orderBy(col("performance_timeobj").cast("timestamp").cast("long")).rangeBetween(-1 * 3600, 0)
#print df.withColumn("lagCheck", F.first(df["performance_cost"]).over(wSpec4)).show()
print df.withColumn("lagCheck", F.first(df["performance_cost"]).over(wSpec4) - df["performance_cost"]).show()


In [ ]:
#Test window query out of range values

from pyspark.sql.functions import coalesce

#Filtered date windowed query
wSpec3 = Window.partitionBy(partitioning_list).orderBy(col("performance_timeobj").cast("timestamp").cast("long")).rangeBetween(-1 * 3600, -1)
df2 = df.withColumn("cumulativeDateFilteredSum", coalesce(df["bq_resale_orders"] - sum(df["bq_resale_orders"]).over(wSpec3),df["bq_resale_orders"]))
print df2.show()
df2.repartition(1).write.format('json').save('/tmp/tmp.json')



In [ ]:
# column coalesce to replace null values
# http://stackoverflow.com/questions/42990533/pyspark-replace-null-in-column-with-value-in-other-column

from pyspark.sql.functions import coalesce
cDf = sqlContext.createDataFrame([(None, None), (1, None), (None, 2),(3,4)], ("a", "b"))
cDf.show()
print cDf.select(coalesce(cDf["a"], cDf["b"])).show()


In [ ]:
import sys
print -sys.maxint
from datetime import datetime, timedelta

d = datetime.strptime('2016-02-02 01:02:12', "%Y-%m-%d %H:%M:%S") 

print d - timedelta(days=7)
print d - timedelta(days=1)
print d - timedelta(hours=1), type(d - timedelta(hours=1))

In [ ]:
#Spark mapparitions tinkering

def my_print_f(iterator):
    if iterator:
        print 'yo:'
        for r in iterator:
            print r
    print '****'
    
def my_print_f(x):
    print 'yo:'
    print r
    print '****'

d = sc.parallelize([('ab',1,10),('ab',2,11),('qb',3,4),('qb',5,7),('a',1,1),('a',0,2)],3)
#d = sc.parallelize([1,2,3,3,4,5,6,6,7,8,8,9],3)
df = sqlContext.createDataFrame(d,['grp','order','val'])
df = df.orderBy(['grp']).sortWithinPartitions("order")
print df.rdd.getNumPartitions()
#print df.show()
#df.rdd.foreachPartition(print_f)
#print d.getNumPartitions()
#print d.collect()
# print df.rdd.mapPartitions(my_print_f, preservesPartitioning=True).collect()
#print df.rdd.foreachPartition(my_print_f)
# pixel_merged_rdd = ord_df.rdd.mapPartitions(process_keyword_partition, preservesPartitioning=True)

print df.rdd.map(my_print_f).collect()

In [ ]:
#Spark window query based join

customers = sc.parallelize([(1234, 1234, datetime.strptime('2016-02-02 00:01:12', "%Y-%m-%d %H:%M:%S"), 1, 110, 1),
                            (1234, 1234, datetime.strptime('2016-02-02 02:02:12', "%Y-%m-%d %H:%M:%S"), 1, 111, 2),
                            (1234, 1234, datetime.strptime('2016-02-02 03:02:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 3),
                            (1235, 1235, datetime.strptime('2016-02-02 01:02:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 5),
                            (1235, 1235, datetime.strptime('2016-02-02 02:02:12', "%Y-%m-%d %H:%M:%S"), 1, 113, 10),
                            (1236, 1236, datetime.strptime('2016-02-02 01:02:12', "%Y-%m-%d %H:%M:%S"), 1, 114, 14),
                            (1236, 1236, datetime.strptime('2016-02-02 03:02:12', "%Y-%m-%d %H:%M:%S"), 1, 115, 22)])

customers2 = sc.parallelize([(1234, 1234, datetime.strptime('2016-02-02 00:02:12', "%Y-%m-%d %H:%M:%S"), 1, 110, 1),
                            (1234, 1234, datetime.strptime('2016-02-02 00:03:12', "%Y-%m-%d %H:%M:%S"), 1, 111, 2),
                            (1234, 1234, datetime.strptime('2016-02-02 02:03:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 3),
                            (1235, 1235, datetime.strptime('2016-02-02 01:03:12', "%Y-%m-%d %H:%M:%S"), 1, 112, 5),
                            (1235, 1235, datetime.strptime('2016-02-02 02:01:12', "%Y-%m-%d %H:%M:%S"), 1, 113, 10),
                            (1236, 1236, datetime.strptime('2016-02-02 01:03:12', "%Y-%m-%d %H:%M:%S"), 1, 114, 14),
                            (1236, 1236, datetime.strptime('2016-02-02 03:01:12', "%Y-%m-%d %H:%M:%S"), 1, 115, 22)])

df = hiveContext.createDataFrame(customers, ["performance_campaignId", "performance_keywordId", "performance_timeobj", "bq_resale_orders", "bq_resale_gtv", "performance_cost"])
df2 = hiveContext.createDataFrame(customers2, ["performance_campaignId", "performance_keywordId", "performance_timeobj", "bq_resale_orders", "bq_resale_gtv", "performance_cost"])

print df.printSchema()

partitioning_list = ["performance_campaignId","performance_keywordId"]

wSpec5 = Window.partitionBy(partitioning_list).orderBy(col("performance_timeobj"))
wSpec6 = Window.partitionBy(partitioning_list).orderBy(col("performance_timeobj").cast("timestamp").cast("long")).rangeBetween(-1 * 3600, 0)
df = df.withColumn("timestamplong", df['performance_timeobj'].cast("timestamp").cast("long"))
df2 = df2.withColumn("timestamplong", df2['performance_timeobj'].cast("timestamp").cast("long"))
df = df.withColumn("lagCheck", F.lag(df["timestamplong"]).over(wSpec5))
#df = df.withColumn("aggVal",F.sum(df2["bq_resale_orders"]).over(Window.partitionBy(partitioning_list).orderBy("timestamplong").rangeBetween(-1 * df['lagCheck'], 0)))

df.rdd.map(my_agg(df2))

print df.show()

In [ ]:
#Check partition lengths skew
#http://stackoverflow.com/questions/28687149/how-to-get-the-number-of-elements-in-partition

num_partitions = 20000
a = sc.parallelize(range(int(1e6)), num_partitions)
l = a.glom().map(len).collect()  # get length of each partition
print(min(l), max(l), sum(l)/len(l), len(l))  # check if skewed


In [ ]:
#Test reduceByKey
from operator import add
def reduce_func(accum_x, x):
    if accum_x is None:
        print 'accum_x none, x:', x
    if x is None:
        print 'x none, accum_x:', accum_x
    if accum_x is None:
        accum_x = x
    else:     
        accum_x = map(add,accum_x,x)
    return accum_x

b = sc.parallelize([(1,1,1,10), (3,1,1,2), (1,1,2,2), (2,1,2,2), (2,1,2,1), (1,2,1,1), (1,2,1,2), (3,1,2,1) ])
#print b.map(lambda x : (x[0],x)).collect()
#print b.map(lambda x : (x[0],x[0])).groupByKey().reduceByKey(reduce_func).collect()
print b.map(lambda x : (x[0],x)).reduceByKey(reduce_func).collect()


In [ ]:
#Element wise addition
from operator import add
a=[1,2]
b=[2,3]
print map(add,a,b)

In [ ]:
#Joining 2 dataframes
customers1 = sc.parallelize([(1234, 2, 10, 110, 1),
                            (1235, 1234, 1, 111, 2),
                            (1236, 1234, 1, 112, 3),
                            (1237, 1235, 1, 112, 5),
                            (1238, 1235, 1, 113, 10),
                            (1239, 1236, 1, 114, 14),
                            (1240, 7, 11, 115, 22)])

customers2 = sc.parallelize([(1234, 2, 1, 110, 1),
                            (1233, 3, 1, 111, 2),
                            (1232, 4, 1, 112, 3),
                            (1231, 5, 1, 112, 5),
                            (1237, 6, 1, 113, 10),
                            (1240, 7, 1, 114, 14),
                            (1290, 8, 1, 115, 22)])

df1 = sqlContext.createDataFrame(customers1, ["a1","b1","c1","d1","e1"])
df2 = sqlContext.createDataFrame(customers2, ["a2","b2","c2","d2","e2"])

#Approach1
df3 = df1.join(df2, (df1.a1 == df2.a2) & (df1.b1 == df2.b2))
print df3.show()

#Approach2
df1.registerTempTable("df1")
df2.registerTempTable("df2")
print sqlContext.sql("SELECT *  from df1, df2 where df1.a1 = df2.a2 AND df1.b1 = df2.b2").show()

# #Approach1 variant
# join_cond = (df1.a1 == df2.a2) & (df1.b1 == df2.b2)
# print join_cond
# df4 = df1.join(df2, join_cond, 'left_outer')
# join_cond = (df4.a1 == df2.a2) & (df4.b1 == df2.b2)
# df5 = df4.join(df2, join_cond, 'left_outer')
# print df5.show()


In [ ]:
#Pyspark reduce_by hack

#https://codereview.stackexchange.com/questions/115082/generic-reduceby-or-groupby-aggregate-functionality-with-spark-dataframe

from pyspark.sql import Row
from pyspark.sql.functions import struct
from pyspark.sql import DataFrame
from collections import OrderedDict

def reduce_by(self, by, cols, f, schema=None):
    """
    :param self DataFrame
    :param by a list of grouping columns 
    :param cols a list of columns to aggregate
    :param aggregation function Row => Row
    :return DataFrame
    """
    def merge_kv(kv):
        key, value = kv
        return Row(**OrderedDict(zip(
            key.__fields__ + value.__fields__, key + value)
        ))

    return (self
        .select(struct(*by), struct(*cols))
        .rdd
        .reduceByKey(f)
        .map(merge_kv)
        .toDF(schema))

DataFrame.reduce_by = reduce_by  # A quick monkey patch

def foo(row1, row2):
    """ A dummy function
    >>> foo(Row(x=1, y=None), Row(x=None, y=2))
    Row(x=1, y=2)
    """
    return Row(**OrderedDict(zip(
      row1.__fields__, (x if x else y for (x, y) in zip(row1, row2))
    )))

# Example data
df = sc.parallelize([
    ("a", None, 1), ("a", None, 2), ("a", 3, None),
    ("b", None, 2), ("b", None, None), ("c", 2, -2), ("c", 1, -1)
]).toDF(["k", "v1", "v2"])

df.reduce_by(by=["k"], cols=["v1", "v2"], f=foo).show()

In [ ]:
#Updating dataframe column names

#https://stackoverflow.com/questions/34077353/how-to-change-dataframe-column-names-in-pyspark

oldColumns = df.schema.names
newColumns = map(lambda x : 'abc_' + x, oldColumns)
#df.schema.names = newColumns
print oldColumns, newColumns

df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)), df)
df.printSchema

df.show()

In [ ]:
#udf on multiple columns

customers1 = sc.parallelize([(1234, 2, 10, 110, 1),
                            (1235, 1234, 1, 111, 2),
                            (1236, 1234, 1, 112, 3),
                            (1237, 1235, 1, 112, 5),
                            (1238, 1235, 1, 113, 10),
                            (1239, 1236, 1, 114, 14),
                            (1240, 7, 11, 115, 22)])

df1 = sqlContext.createDataFrame(customers1, ["a1","b1","c1","d1","e1"])

udf_add = udf(lambda x,y : x+y, IntegerType())

df2 = df1\
    .withColumn("f1", udf_add(col("a1"),col("b1")))
    
df2.show()

In [ ]:
#Inspect read from s3

file_regex='s3://a/b*.csv'
from pyspark.sql import SparkSession

# spark = SparkSession \
#     .builder \
#     .appName("Python Spark SQL basic example") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()
    
# full_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').option("escape","\"").load(file_regex)


In [ ]:
a = [1,2]
b = [3,4]
b = [3,4]
a+b
print a+b, a, b

In [ ]:
#Model error metrics

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

def mean_absolute_percentage_error(y_true, y_pred): #Make sure no values are 0
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mean_ape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mean_ape

def median_absolute_percentage_error(y_true, y_pred): #Make sure no values are 0
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    median_ape = np.median(np.abs((y_true - y_pred) / y_true)) * 100
    return median_ape

def compute_rmse(y_true, y_pred): 
    cur_rmse = sqrt(mean_squared_error(y_true, y_pred))
    return cur_rmse

def evaluate_predictions(y_true, y_pred, cur_scenario, sample_size=0, predictor_size=0):
    y_pred = map(lambda x : float(x), y_pred)
    cur_mean_ape = mean_absolute_percentage_error(y_true, y_pred)
    cur_median_ape = median_absolute_percentage_error(y_true, y_pred)
    cur_rmse = compute_rmse(y_true, y_pred)
    cur_mae = mean_absolute_error(y_true, y_pred)
    cur_r2_score = r2_score(y_true, y_pred)
    adjusted_r_squared = 1 - (1-cur_r2_score)*(sample_size-1)/(sample_size-predictor_size-1)
    print 'Mean abs percent err | RMSE | Median abs percent err | Mean abs err | R2 score | Adjusted R2 score | Model |'
    print '%s | %s | %s | %s | %s |%s | %s |'%(cur_mean_ape, cur_rmse, cur_median_ape, cur_mae, cur_r2_score, adjusted_r_squared, cur_scenario)
    return cur_median_ape, cur_rmse


# Spark Machine Learning

In [ ]:
# https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression
# https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/regression/RandomForestRegressor.scala
# http://blog.learningtree.com/how-to-predict-outcomes-using-random-forests-and-spark/
# https://spark.apache.org/docs/latest/mllib-data-types.html
# https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/2485090270202665/2400794110066361/8589256059752547/latest.html
#https://spark.apache.org/docs/latest/ml-features.html (Use Vector assembler to group diff features into 1 group if required)

# from pyspark.sql.types import *
# categorical_cols = ['a']
# numerical_cols = ['b','c','label']
# categorical_fields = [StructField(field_name, StringType(), True) for field_name in categorical_cols]
# numerical_fields = [StructField(field_name, DoubleType(), True) for field_name in numerical_cols]
# all_fields = categorical_fields + numerical_fields
# schema = StructType(all_fields)
# df = sqlContext.createDataFrame(rdd, schema)

#Spark ml RandomForest Regressor

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline

from pyspark.ml.linalg import Vectors
def parse_ml(l):
    return (l[3], Vectors.dense(l[1:3]))

cur_rdd = sc.parallelize([("a",1.0,1.0,10.2), ("b",-1.0,1.0,2.0), ("a",1.0,2.0,22.5), ("b",-5.0,2.0,2.0), ("b",-2.0,2.0,1.0), ("b",-22.0,1.0,1.9), ("a",2.0,1.0,12.2), ("b",-11.0,2.0,1.0) ])
ml_rdd = cur_rdd.map(parse_ml)
new_df = sqlContext.createDataFrame(ml_rdd, ["label", "features"])

# Train a RandomForest model
rf = RandomForestRegressor(numTrees=4, maxDepth=10)
# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[rf])
# Train model.  This also runs the (indexer) other stages
model = rf.fit(new_df) #model = pipeline.fit(new_df). Ignore pipeline for now for .featureImportances to work
# Make predictions
Y_predictions = model.transform(new_df)

y_pred = Y_predictions.select('prediction').toPandas().values
y_true = np.array([10.2,2.0,22.5,2.0,1.0,1.9,12.2,1.0])
evaluate_predictions(y_true, y_pred, 'Spark ML RFR')

print model.featureImportances
    

In [ ]:
#Spark mllib RandomForest Regressor

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils


cat_num_map = {"a":0, "b":1}

from pyspark.mllib.feature import LabeledPoint
def parse_mllib(l):
    return LabeledPoint(l[3], l[1:3]) #list(cat_num_map[l[0]]) + l[1:3])

cur_rdd = sc.parallelize([("a",1,1.0,10.2), ("b",-1.0,1.0,2.0), ("a",1.0,2.0,22.5), ("b",-5.0,2.0,2.0), ("b",-2.0,2.0,1.0), ("b",-22.0,1.0,1.9), ("a",2.0,1.0,12.2), ("b",-11.0,2.0,1.0) ])
mllib_rdd = cur_rdd.map(parse_mllib) #label, features (implicit headers)

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainRegressor(mllib_rdd, categoricalFeaturesInfo={},
                                    numTrees=3, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(mllib_rdd.map(lambda x : x.features))
print(model.toDebugString())

y_pred = np.array(predictions.collect())
y_true = np.array(mllib_rdd.map(lambda x : x.label).collect())
evaluate_predictions(y_true, y_pred, 'Spark MLLib RFR')
